In [1]:
!pip install sentence-transformers numpy openai requests beautifulsoup4

In [2]:
import json
import numpy as np
from typing import List, Dict
from sentence_transformers import SentenceTransformer

class DataPreparation:
    def __init__(self, chunk_size=100, chunk_overlap=20):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.chunks = []

    def process_documents(self, documents: List[str]):
        """Разбиваем документы на фрагменты"""
        print("Разбиваем документы на фрагменты.....")
        for doc in documents:
            # Простое разбиение по словам
            words = doc.split()
            for i in range(0, len(words), self.chunk_size - self.chunk_overlap):
                chunk_text = " ".join(words[i:i + self.chunk_size])
                self.chunks.append({
                    'text': chunk_text,
                    'source': f"Документ {documents.index(doc) + 1}",
                    'chunk_id': len(self.chunks)
                })
        print(f"Создано {len(self.chunks)} фрагментов")
        return self.chunks

    def create_embeddings(self, model_name='all-MiniLM-L6-v2'):
        """Создаем векторные представления"""
        print("Создаем векторные представления.......")
        model = SentenceTransformer(model_name)
        texts = [chunk['text'] for chunk in self.chunks]
        vectors = model.encode(texts)
        print(f"Создано {len(vectors)} векторов размерности {vectors.shape[1]}")
        return vectors

    def save_knowledge_base(self, base_path='knowledge_base'):
        """Сохраняем базу знаний"""
        vectors = self.create_embeddings()

        # Сохраняем фрагменты
        with open(f'{base_path}_chunks.json', 'w', encoding='utf-8') as f:
            json.dump(self.chunks, f, ensure_ascii=False, indent=2)

        # Сохраняем векторы
        np.save(f'{base_path}_vectors.npy', vectors)  # Исправлено: vectors вместо vector
        print(f"База знаний сохранена в файл {base_path}_*")

class SearchSystem:
    def __init__(self, base_path='knowledge_base', embedding_model='all-MiniLM-L6-v2'):
        # Загружаем модель для эмбеддингов
        self.model = SentenceTransformer(embedding_model)

        # Загружаем фрагменты
        try:
            with open(f'{base_path}_chunks.json', 'r', encoding='utf-8') as f:
                self.chunks = json.load(f)
        except FileNotFoundError:
            print(f"Файл {base_path}_chunks.json не найден!")
            self.chunks = []

        # Загружаем векторы
        try:
            self.vectors = np.load(f'{base_path}_vectors.npy')  # Исправлено: vectors вместо vector
        except FileNotFoundError:
            print(f"Файл {base_path}_vectors.npy не найден!")
            self.vectors = np.array([])

        print(f'Загружена база знаний: {len(self.chunks)} фрагментов')

    def find_similar(self, query: str, top_k=3) -> List[Dict]:
        """Находим наиболее релевантные фрагменты"""
        if len(self.vectors) == 0:
            return []

        # Векторизуем запрос
        query_vector = self.model.encode([query])

        # Вычисляем косинусное сходство
        similarities = np.dot(self.vectors, query_vector.T).flatten()
        base_norms = np.linalg.norm(self.vectors, axis=1)
        query_norm = np.linalg.norm(query_vector)
        cosine_similarities = similarities / (base_norms * query_norm)

        # Находим индексы наиболее похожих фрагментов
        top_indices = np.argsort(cosine_similarities)[-top_k:][::-1]

        # Формируем результат
        results = []
        for idx in top_indices:
            result = self.chunks[idx].copy()
            result['similarity'] = float(cosine_similarities[idx])
            results.append(result)

        return results

class AnswerGenerator:
    def __init__(self, api_key: str):
        self.api_key = api_key

    def get_answer(self, question: str, chunks: List[Dict]) -> str:
        """Генерируем ответ на основе найденных фрагментов"""
        # Имитация генерации ответа (в реальности здесь был бы вызов LLM API)
        context = "\n".join([chunk['text'] for chunk in chunks])
        return f"Ответ основан на контексте:\n{context}\n\nВопрос: {question}"

class RAGSystem:
    def __init__(self, api_key: str, base_path='knowledge_base'):  # Исправлено: base_path вместо base_patch
        self.search = SearchSystem(base_path)
        self.generator = AnswerGenerator(api_key)

    def answer(self, question: str, num_chunks=3) -> Dict:
        """Основная функция поиск + генерация ответа"""
        print(f'Обрабатываем вопрос: {question}')

        # Находим релевантные фрагменты
        chunks = self.search.find_similar(question, num_chunks)
        print('\nНайденные фрагменты: ')
        for i, chunk in enumerate(chunks, 1):
            similarity = int(chunk['similarity'] * 100)
            preview = chunk['text'][:100] + '...'
            print(f'{i}. Сходство {similarity}%: {preview}')

        # Генерируем ответ
        answer = self.generator.get_answer(question, chunks)

        return {
            'question': question,
            'answer': answer,
            'sources': chunks,
            'num_sources': len(chunks)
        }

    def ask(self, question: str, num_chunks=3) -> Dict:  # Добавлен метод ask
        return self.answer(question, num_chunks)

if __name__ == '__main__':
    documents = [
        'Python - высокоуровневый язык программирования общего назначения. Создан в 1991 году Гвидо ван Россумом.',
        'RAG - архитектурный подход, который объединяет поиск информации с генерацией текста.',
        'Векторные базы данных хранят данные в виде многомерных векторов и позволяют выполнять семантический поиск.',
        'FAISS - библиотека для эффективного поиска схожих векторов.',
        'Эмбеддинги - это векторные представления текста, которые кодируют семантическое значение в числовой форме.'
    ]

    # Создаем и сохраняем базу знаний
    data_prep = DataPreparation()
    data_prep.process_documents(documents)
    data_prep.save_knowledge_base()

    # Создаем RAG систему
    rag = RAGSystem(api_key='sk-1234abcd1234abcd1234abcd1234abcd1234abcd')

    # Задаем вопросы
    questions = [
        'Кто создал Python?',
        'Что такое RAG?',
        'Как работают векторные базы данных?',
        'Что такое FAISS?'
    ]

    for question in questions:
        result = rag.ask(question)
        print(f"Вопрос: {result['question']}")
        print(f"Ответ: {result['answer']}")
        print(f"Источники: {len(result['sources'])} фрагментов")
        print("-" * 60)

Разбиваем документы на фрагменты.....
Создано 5 фрагментов
Создаем векторные представления.......


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Создано 5 векторов размерности 384
База знаний сохранена в файл knowledge_base_*
Загружена база знаний: 5 фрагментов
Обрабатываем вопрос: Кто создал Python?

Найденные фрагменты: 
1. Сходство 60%: Python - высокоуровневый язык программирования общего назначения. Создан в 1991 году Гвидо ван Россу...
2. Сходство 49%: RAG - архитектурный подход, который объединяет поиск информации с генерацией текста....
3. Сходство 47%: FAISS - библиотека для эффективного поиска схожих векторов....
Вопрос: Кто создал Python?
Ответ: Ответ основан на контексте:
Python - высокоуровневый язык программирования общего назначения. Создан в 1991 году Гвидо ван Россумом.
RAG - архитектурный подход, который объединяет поиск информации с генерацией текста.
FAISS - библиотека для эффективного поиска схожих векторов.

Вопрос: Кто создал Python?
Источники: 3 фрагментов
------------------------------------------------------------
Обрабатываем вопрос: Что такое RAG?

Найденные фрагменты: 
1. Сходство 61%: RAG - архитек